In [1]:
%cd ..

/home/eli/AnacondaProjects/ppc_experiments


In [2]:
import argparse
import collections
import numpy as np
import pyro
import torch
import logger, train

Seed set to 123


In [3]:
# pyro.enable_validation(True)
# torch.autograd.set_detect_anomaly(True)

In [4]:
config, (data, model, trainer) = train.from_file("experiments/ppc_mnist_config.json")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [5]:
logger = config.get_logger("train")
logger.info(config.log_dir)

saved/log/Mnist_Ppc/0508_163004


In [6]:
trainer.fit(model, data)

Missing logger folder: saved/log/Mnist_Ppc/0508_163004/lightning_logs
/home/eli/miniforge3/envs/ppc/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /home/eli/AnacondaProjects/ppc_experiments/saved/models/Mnist_Ppc/0508_163004 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | graph      | MnistPpc   | 319 K 
1 | predictive | Predictive | 0     
------------------------------------------
319 K     Trainable params
0         Non-trainable params
319 K     Total params
1.278     Total estimated model params size (MB)


Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=10` reached.


In [7]:
trainer.fit(model, data, ckpt_path=str(config.save_dir) + "/checkpoint_9.ckpt")

Restoring states from the checkpoint path at saved/models/Mnist_Ppc/0508_163004/checkpoint_9.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | graph      | MnistPpc   | 319 K 
1 | predictive | Predictive | 0     
------------------------------------------
319 K     Trainable params
0         Non-trainable params
319 K     Total params
1.278     Total estimated model params size (MB)
Restored all states from the checkpoint at saved/models/Mnist_Ppc/0508_163004/checkpoint_9.ckpt


Sanity Checking: |                                                                                            …

`Trainer.fit` stopped: `max_epochs=10` reached.
